In [1]:
from typing import List, Tuple

import numpy as np
from scipy import stats

In [2]:
N = 6
S = 8
I1 = 5
I2 = 4

print(f"""
N = {N}
S = {S}
I1 = {I1}
I2 = {I2}
""")


N = 6
S = 8
I1 = 5
I2 = 4



### Task A


In [3]:
c = S
d = 2 + N
sample_size = (1 + N) * 10
B = 100 * sample_size
gamma = 0.9
alpha = 1 - gamma
print(f"""
c = {c}
d = {d}
sample_size = {sample_size}
B = {B}
gamma = {gamma}
alpha = {alpha}
""")


c = 8
d = 8
sample_size = 70
B = 7000
gamma = 0.9
alpha = 0.09999999999999998



In [4]:
def _format_ci(ci: List) -> Tuple[float, float, float]:
    ci_min = ci[0].item()
    ci_max = ci[1].item()
    ci_range = ci_max - ci_min
    return ci_min, ci_max, ci_range


def compute_ci(bootstrap_distribution: List, observed_statistic: float):
    std_error = np.std(bootstrap_distribution)
    z = stats.norm.ppf(1 - alpha / 2)

    # Normal CI
    normal_ci = [observed_statistic - z * std_error, observed_statistic + z * std_error]

    # Percentile CI
    lower_percentile = np.percentile(bootstrap_distribution, 100 * alpha / 2)
    upper_percentile = np.percentile(bootstrap_distribution, 100 * (1 - alpha / 2))
    percentile_ci = [lower_percentile, upper_percentile]

    # Pivotal CI
    pivotal_ci = [
        2 * observed_statistic - upper_percentile,
        2 * observed_statistic - lower_percentile,
    ]
    print("\tNormal CI: min {} max {} range {}".format(*_format_ci(normal_ci)))
    print("\tPercentile CI: min {} max {} range {}".format(*_format_ci(percentile_ci)))
    print("\tPivotal CI: min {} max {} range {}".format(*_format_ci(pivotal_ci)))

In [5]:
Y_N = stats.pareto.rvs(d, scale=c, size=sample_size)
shape, _, scale = stats.pareto.fit(Y_N, floc=0)
observed_variance = np.var(Y_N, ddof=1) / N

p_vars, np_vars, mc_vars = [], [], []
for _ in range(B):
    # Parametric bootstrap
    p_sample = stats.pareto.rvs(shape, scale=scale, size=sample_size)
    p_vars.append(np.var(p_sample, ddof=1) / N)
    # Non-parametric bootstrap
    np_sample = np.random.choice(Y_N, size=sample_size, replace=True)
    np_vars.append(np.var(np_sample, ddof=1) / N)
    # Monte Carlo
    mc_sample = stats.pareto.rvs(b=d, scale=c, size=sample_size)
    mc_vars.append(np.var(mc_sample, ddof=1) / N)

print("Parametric")
compute_ci(p_vars, observed_variance)
print("Non-parametric")
compute_ci(np_vars, observed_variance)
print("Monte Carlo")
compute_ci(mc_vars, observed_variance)

Parametric
	Normal CI: min -0.02935016655775141 max 0.6692018472432748 range 0.6985520138010262
	Percentile CI: min 0.1612965184333312 max 0.7305368700558147 range 0.5692403516224835
	Pivotal CI: min -0.09068518937029124 max 0.47855516225219225 range 0.5692403516224835
Non-parametric
	Normal CI: min 0.15747716749057136 max 0.48237451319495206 range 0.32489734570438067
	Percentile CI: min 0.16108783494252762 max 0.48372105651809266 range 0.32263322157556507
	Pivotal CI: min 0.1561306241674308 max 0.4787638457429958 range 0.322633221575565
Monte Carlo
	Normal CI: min 0.04276466095428394 max 0.5970870197312395 range 0.5543223587769555
	Percentile CI: min 0.13080661247109274 max 0.5691560626042633 range 0.4383494501331706
	Pivotal CI: min 0.07069561808126013 max 0.5090450682144307 range 0.4383494501331706


### Task B

In [6]:
Y_N = stats.pareto.rvs(d, scale=c, size=sample_size)
observed_ml_estimator = min(Y_N)

p_vars, np_vars, mc_vars = [], [], []
for _ in range(B):
    # Parametric bootstrap
    p_sample = stats.pareto.rvs(shape, scale=scale, size=sample_size)
    p_vars.append(np.min(p_sample))
    # Non-parametric bootstrap
    np_sample = np.random.choice(Y_N, size=sample_size, replace=True)
    np_vars.append(np.min(np_sample))
    # Monte Carlo
    mc_sample = stats.pareto.rvs(b=d, scale=c, size=sample_size)
    mc_vars.append(np.min(mc_sample))

p_var = np.var(p_vars, ddof=1)
np_var = np.var(np_vars, ddof=1)
mc_var = np.var(mc_vars, ddof=1)

print("Parametric")
compute_ci(p_vars, observed_ml_estimator)
print("Non-parametric")
compute_ci(np_vars, observed_ml_estimator)
print("Monte Carlo")
compute_ci(mc_vars, observed_ml_estimator)

Parametric
	Normal CI: min 7.980539210729824 max 8.03038982234589 range 0.04985061161606641
	Percentile CI: min 8.014862044782019 max 8.05964108546149 range 0.04477904067947058
	Pivotal CI: min 7.951287947614226 max 7.9960669882936966 range 0.04477904067947058
Non-parametric
	Normal CI: min 7.992991242096209 max 8.017937790979508 range 0.024946548883298725
	Percentile CI: min 8.005464516537858 max 8.009439559325434 range 0.003975042787576655
	Pivotal CI: min 8.001489473750281 max 8.005464516537858 range 0.003975042787576655
Monte Carlo
	Normal CI: min 7.981653687750929 max 8.029275345324788 range 0.04762165757385883
	Percentile CI: min 8.000719995933395 max 8.042582556498509 range 0.04186256056511439
	Pivotal CI: min 7.9683464765772065 max 8.010209037142321 range 0.04186256056511439
